# Sentiment Analysis with IMDB dataset using Pytorch

In [1]:
import numpy as np
import keras
from keras.datasets import imdb
import matplotlib.pyplot as plt
import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
%matplotlib inline

np.random.seed(42)

Using TensorFlow backend.


# Dataset Prepration

In [3]:
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# call load_data with allow_pickle implicitly set to true
# Loading the data (it's preloaded in Keras)
(x_train, y_train), (x_test, y_test), = imdb.load_data(num_words=1000)
# restore np.load for future normal usage
np.load = np_load_old

print(x_train.shape)
print(x_test.shape)

(25000,)
(25000,)
